# Preparing

In [1]:
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')

root_dir = '/content/drive/My Drive/'

%cd "{root_dir}"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive


In [2]:
repo_dir = "/content/repo"
!git clone https://github.com/xelav/Insider-Threat-CERT "$repo_dir"

import sys
sys.path.append(repo_dir)

Cloning into '/content/repo'...
remote: Enumerating objects: 253, done.
remote: Counting objects: 100% (253/253), done.
remote: Compressing objects: 100% (141/141), done.
remote: Total 253 (delta 170), reused 183 (delta 110), pack-reused 0
Receiving objects: 100% (253/253), 1.40 MiB | 1.16 MiB/s, done.
Resolving deltas: 100% (170/170), done.


In [3]:
!pip install pytorch-ignite
%pip install wandb -q

import wandb
wandb.login()

     |████████████████████████████████| 112kB 12.2MB/s 
     |████████████████████████████████| 1.4MB 23.8MB/s 
     |████████████████████████████████| 112kB 60.6MB/s 
     |████████████████████████████████| 460kB 60.5MB/s 
     |████████████████████████████████| 102kB 15.8MB/s 
     |████████████████████████████████| 102kB 15.8MB/s 
     |████████████████████████████████| 71kB 11.3MB/s 
     |████████████████████████████████| 71kB 12.4MB/s 


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [1]:
%cd "{repo_dir}"
!git reset --hard
!git checkout new-cnn-parameters
!git fetch
!git reset origin/new-cnn-parameters --hard
%cd "{root_dir}"

[Errno 2] No such file or directory: '{repo_dir}'
/content
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
[Errno 2] No such file or directory: '{root_dir}'
/content


In [32]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

from pathlib import Path
import pandas as pd
import pickle
import numpy as np
import shutil
from tqdm import tqdm

import torch.utils.data
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator, Engine
from ignite.metrics import Accuracy, Loss

from src.models import InsiderClassifier, LSTM_Encoder, LSTM_Encoder_Topics
from src.params import get_params
from src.dataset import CertDataset, create_data_loaders
from src.lstm_trainer import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Prepare run config



In [0]:
# output_dir = Path(r'C:\Users\Mvideo\Google Drive\Datasets\CERT_output')
# answers_dir = Path(r"C:/Users/Mvideo/Downloads/answers")

output_dir = Path(f'{root_dir}/Datasets/CERT_output')
answers_dir = Path(f"{root_dir}/Datasets/CERT/answers")
main_answers_file = answers_dir / "insiders.csv"

assert(output_dir.is_dir())
assert(answers_dir.is_dir())

# log_dir = output_dir / 'logs' / run_name
# checkpoint_dir = output_dir / 'checkpoints' / run_name

# run_name = 'lstm/content-test7'

# assert(not log_dir.is_dir())
# assert(not checkpoint_dir.is_dir())

# if log_dir.is_dir():
#     shutil.rmtree(log_dir)
# if checkpoint_dir.is_dir():
#     shutil.rmtree(checkpoint_dir)

In [0]:
params = get_params()
params['model']['lstm_encoder']['embedding_size'] = None
params['model']['lstm_encoder']['use_content_topics'] = False

# This is a mess of the code..

# add content topics
# WARNING: don't mix embeddings with content topics!
# params['model']['lstm_encoder']['input_size']

In [0]:
# actions, targets = CertDataset.prepare_dataset(output_dir / 'aggregated.pkl',
#                                                main_answers_file,
#                                                min_length=50,
#                                                max_length=200,
#                                                )

In [0]:
df = pd.read_pickle(root_dir + '/df_final.pkl')

df.action_id = CertDataset.pad_to_length(df.action_id)
actions = np.vstack(df.action_id.values)

content = None
if params['model']['lstm_encoder']['use_content_topics']:
    content_values = df.content.values
    content_values = CertDataset.pad_topic_matricies(content_values, max_length=200)
    content = df.content

cert_dataset = CertDataset(actions, df.malicious, content_topics=content)
train_loader, val_loader = create_data_loaders(cert_dataset, validation_split=0.3, random_seed=0, batch_size=1024)

device = 'cuda'

# Prepare trainers

In [36]:
%cd /content

/content


In [37]:
wandb.init(project="cert-lstm")
checkpoint_dir = Path(wandb.run.dir) / 'checkpoints'

if params['model']['lstm_encoder']['use_content_topics']:
    LSTM_model = LSTM_Encoder_Topics
else:
    LSTM_model = LSTM_Encoder
lstm_encoder = LSTM_model(params['model']['lstm_encoder'])
criterion = nn.NLLLoss()
optimizer = optim.Adam(lstm_encoder.parameters())

train_engine = create_supervised_trainer_lstm(
                                        lstm_encoder, optimizer, criterion, device=device,
                                        prepare_batch=prepare_batch_lstm,
                                        checkpoint_dir=checkpoint_dir,
                                       )

val_engine = create_supervised_evaluator_lstm(
        lstm_encoder, device=device,
        prepare_batch=prepare_batch_lstm,
        metrics={},
        criterion=criterion,
        checkpoint_dir=checkpoint_dir,
)

@train_engine.on(Events.STARTED)
def log_training_results(trainer):
    print('Initial validation run:')
    val_engine.train_epoch = 0
    val_engine.run(val_loader)

    wandb.run.summary["best_val_non_pad_accuracy"] = val_engine.state.metrics['non_pad_accuracy']
    wandb.run.summary["best_val_loss"] = val_engine.state.metrics['epoch_loss']

@train_engine.on(Events.EPOCH_COMPLETED)
def log_training_results(trainer):
    val_engine.train_epoch = train_engine.state.epoch
    val_engine.run(val_loader)

    if val_engine.state.metrics['non_pad_accuracy'] > wandb.run.summary["best_val_non_pad_accuracy"]:
        wandb.run.summary["val_non_pad_accuracy"] = val_engine.state.metrics['non_pad_accuracy']
        wandb.run.summary["best_val_loss"] = val_engine.state.metrics['epoch_loss']
        wandb.run.summary["best_train_non_pad_accuracy"] = train_engine.state.metrics['non_pad_accuracy']
        wandb.run.summary["best_train_loss"] = train_engine.state.metrics['epoch_loss']

        wandb.save(str(checkpoint_dir) + '/best_model*')


Load word2vec embeddings

In [0]:
pretrained_path = output_dir / 'checkpoints' / 'w2v/test2/final_model_7758.pth'

pretrained_embeddings = torch.load(pretrained_path)
pretrained_embeddings = pretrained_embeddings['embed.weight']

lstm_encoder.embedding = lstm_encoder.embedding.from_pretrained(pretrained_embeddings[:-1])
lstm_encoder.embedding.requires_grad = False

# Run experiment

In [38]:
torch.manual_seed(train_params['manual_seed'])
train_engine.run(train_loader, max_epochs=1000)
print('finished!')

Initial validation run:
Validation Results   - Avg loss: 4.072861, Accuracy: 0.001880, Non-Pad-Accuracy: 0.000593
Epoch 1 completed!
Train Results        - Avg loss: 1.531287, Accuracy: 0.517337, Non-Pad-Accuracy: 0.691417
Validation Results   - Avg loss: 0.911537, Accuracy: 0.729661, Non-Pad-Accuracy: 0.779523
Epoch 2 completed!
Train Results        - Avg loss: 0.621293, Accuracy: 0.852476, Non-Pad-Accuracy: 0.778172
Validation Results   - Avg loss: 0.522705, Accuracy: 0.877446, Non-Pad-Accuracy: 0.777618
Epoch 3 completed!
Train Results        - Avg loss: 0.495538, Accuracy: 0.882222, Non-Pad-Accuracy: 0.777757
Validation Results   - Avg loss: 0.479484, Accuracy: 0.885024, Non-Pad-Accuracy: 0.777503
Epoch 4 completed!
Train Results        - Avg loss: 0.470495, Accuracy: 0.886261, Non-Pad-Accuracy: 0.777686
Validation Results   - Avg loss: 0.464589, Accuracy: 0.887408, Non-Pad-Accuracy: 0.777462
Epoch 5 completed!
Train Results        - Avg loss: 0.460172, Accuracy: 0.887727, Non-Pad-

Current run is terminating due to exception: .
Engine run is terminating due to exception: .


KeyboardInterrupt: ignored

In [25]:
%debug

> /usr/local/lib/python3.6/dist-packages/ignite/handlers/checkpoint.py(167)__call__()
    165         suffix = ""
    166         if self.global_step_transform is not None:
--> 167             global_step = self.global_step_transform(engine, engine.last_event_name)
    168             suffix = "{}".format(global_step)
    169 

ipdb> engine
ipdb> engine.last_event_name
<Events.EPOCH_COMPLETED: 'epoch_completed'>
ipdb> self.global_step_transform(engine, engine.last_event_name)
*** TypeError: <lambda>() takes 1 positional argument but 2 were given
ipdb> self.global_step_transform
<function create_supervised_trainer_lstm.<locals>.<lambda> at 0x7f15ea583bf8>
ipdb> exti
*** NameError: name 'exti' is not defined
ipdb> exit


In [0]:
%ls "{checkpoint_dir}"

'best_model_500_non_pad_accuracy=0.771211054971759.pth'   checkpoint_38000.pth
 checkpoint_37848.pth                                     final_model_38000.pth
 checkpoint_37924.pth


In [0]:
best_model_checkpoint = list(checkpoint_dir.glob('best_model_*'))
assert(len(best_model_checkpoint) == 1)
best_model_checkpoint = best_model_checkpoint[0]

files.download(best_model_checkpoint) 

In [0]:
%load_ext tensorboard
%tensorboard --logdir "{log_dir}"

zip and download logs

In [0]:
log_zip =  output_dir / 'logs' / (run_name + '_logs.zip')
!zip -r "$log_zip" "$log_dir"
files.download(log_zip) 

  adding: content/drive/My Drive/Datasets/CERT_output/logs/lstm/random-embeddings/ (stored 0%)
  adding: content/drive/My Drive/Datasets/CERT_output/logs/lstm/random-embeddings/train/ (stored 0%)
  adding: content/drive/My Drive/Datasets/CERT_output/logs/lstm/random-embeddings/train/events.out.tfevents.1586905646.00b82a3f534b.127.9 (deflated 94%)
  adding: content/drive/My Drive/Datasets/CERT_output/logs/lstm/random-embeddings/validation/ (stored 0%)
  adding: content/drive/My Drive/Datasets/CERT_output/logs/lstm/random-embeddings/validation/events.out.tfevents.1586905646.00b82a3f534b.127.10 (deflated 73%)


# Prediction exploration

TODO:
- decode class numbers

In [0]:
batch = next(iter(val_loader))

In [0]:
x, y = prepare_batch_lstm(batch)

In [0]:
lstm_encoder.train()

In [0]:
y.argmax(dim=2)[2]

In [0]:
lstm_encoder(x.to('cuda')).exp().argmax(dim=2)[2]

In [0]:
lstm_encoder(x.to('cuda')).exp().argmax(dim=2).unique()